# <font color=red>LangChain:  Chains</font>
- https://docs.langchain.com/docs

## What Does LangChain Provide?
+ Models
  + embedding
  + LLM (e.g. OpenAI)
+ Prompts
  + prompt templates
  + few-shot
  + example-selectors
  + output parsers
<span style="font-family:'Comic Sans MS', cursive, sans-serif;"><font color=orange>
+ Chains (a multi-step workflow composed of <em>links</em>)</br>
  + Links (one of: prompt, model, another chain)
</font></span>
+ Vector Database Access
  + Document Loaders
  + Text Splitting 
+ Memories (to facilitate chatbots or other 'iterative' sorts of apps)
+ Agents (loop over Thought, Act, Observe)
  + Tools
    + math
    + web search
    + custom (user-defined)

<span style="font-family:'Comic Sans MS', cursive, sans-serif;"><font color=orange>
## Chains
</font></span>

### A chain is a multi-step workflow, composed of links.
A link is one of:
- a prompt
- an LLM
- another chain
#### Our first chain will be a repeat of the one we had in the Quickstart Guide

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains  import LLMChain
from langchain.chat_models import ChatOpenAI   # switching to a chat model

# note prompt's similarity to python f-strings
template = "Tell me a {joke_type} joke about {topic}."
prompt = PromptTemplate.from_template(template)

# here we are only printing the prompt, not using it with an LLM
# we just want to show that we can make substitutions for the input_variables
print(prompt.format(joke_type="dad",topic="dogs"))

llm = ChatOpenAI(model_name="gpt-4", temperature=0.7, max_tokens=128)

# now, let's create a chain consisting of the prompt and the LLM
llm_chain = LLMChain(prompt=prompt, llm=llm)

response = llm_chain.run(joke_type="dad",topic="my cat named Gizmo")
print(response)

#### Next, a SimpleSequentialChain combining two chains, the second critiquing output of the first

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains  import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.chains import SimpleSequentialChain  # only pass a string between chains

llm = ChatOpenAI(model_name="gpt-4", temperature=0.7, max_tokens=128,)

template = "What is a good Dad joke about {topic}?"
joke_prompt = PromptTemplate.from_template(template)
joke_chain = LLMChain(prompt=joke_prompt, llm=llm)

template = "Is this a good joke: {the_joke}?"
critic_prompt = PromptTemplate.from_template(template)
critic_chain = LLMChain(prompt=critic_prompt, llm=llm)

# chain where we run the two chains in sequence
seq_chain = SimpleSequentialChain(chains=[joke_chain, critic_chain], verbose=True)
critique = seq_chain.run("my cat named Gizmo")
print(critique)

#### <font color=green>LLMMathChain </font>demo to show an example of a specialized built-in chain

In [ ]:
from langchain import LLMMathChain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4", temperature=0)
math_chain = LLMMathChain.from_llm(llm, verbose=True)

response = math_chain.run("What is 13 raised to the .3432 power?")
print(response)

#### We will repeat our first chain here but use LangChain's new Expression Language
The Expression Language permits you to connect links with a pipe symbol, much like in a shell.</br>
It is not totally mature yet, and I still prefer the functional method, but we do it here for completeness.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains  import LLMChain
from langchain.chat_models import ChatOpenAI   # switching to a chat model
from langchain.schema.output_parser import StrOutputParser   # output parser

template = "Tell me a {joke_type} joke about {topic}."
prompt = PromptTemplate.from_template(template)

print(prompt.format(joke_type="dad",topic="dogs"))

llm = ChatOpenAI(model_name="gpt-4", temperature=0.7, max_tokens=128)

llm_chain = prompt | llm | StrOutputParser()    ## <-- the new language syntax

## invoke INSTEAD of run and dictionary of arguments
response = llm_chain.invoke( {"joke_type": "dad", "topic": "my cat named Gizmo"} )
print(response)